In [2]:
from itertools import product

def hamming_distance(s1: str, s2: str) -> int:
    """Computes the Hamming distance between two strings."""
    return sum(1 for a, b in zip(s1, s2) if a != b)

def d(pattern: str, dna: list[str]) -> int:
    """Computes the sum of the minimum Hamming distances between pattern and each string in dna."""
    k = len(pattern)
    total_distance = 0
    
    for seq in dna:
        min_distance = float('inf')
        for i in range(len(seq) - k + 1):
            k_mer = seq[i:i+k]
            min_distance = min(min_distance, hamming_distance(pattern, k_mer))
        total_distance += min_distance
    
    return total_distance

def median_string(dna: list[str], k: int) -> str:
    """Finds the median string of length k that minimizes d(Pattern, Dna)."""
    best_pattern = None
    min_distance = float('inf')
    
    # Generate all possible k-mers
    for pattern in map("".join, product("ACGT", repeat=k)):
        distance = d(pattern, dna)
        if distance < min_distance:
            min_distance = distance
            best_pattern = pattern
    
    return best_pattern

# Sample input
k = 3
dna = ["AAATTGACGCAT", "GACGACCACGTT", "CGTCAGCGCCTG", "GCTGAGCACCGG", "AGTACGGGACAG"]

# Finding the median string
print(median_string(dna, k))


ACG


In [5]:
from collections import Counter

def compute_profile(motifs: list[str], k: int, t: int) -> dict:
    profile = {"A": [0] * k, "C": [0] * k, "G": [0] * k, "T": [0] * k}
    for i in range(k):
        column = [motif[i] for motif in motifs]
        counts = Counter(column)
        for nucleotide in "ACGT":
            profile[nucleotide][i] = counts[nucleotide] / t
    return profile

def profile_most_probable_kmer(text: str, k: int, profile: dict) -> str:
    max_prob = -1
    best_kmer = text[:k]
    for i in range(len(text) - k + 1):
        kmer = text[i:i + k]
        prob = 1
        for j, nucleotide in enumerate(kmer):
            prob *= profile[nucleotide][j]
        if prob > max_prob:
            max_prob = prob
            best_kmer = kmer
    return best_kmer

def score(motifs: list[str], k: int) -> int:
    score = 0
    for i in range(k):
        column = [motif[i] for motif in motifs]
        most_common = Counter(column).most_common(1)[0][1]
        score += len(motifs) - most_common
    return score

def greedy_motif_search(dna: list[str], k: int, t: int) -> list[str]:
    best_motifs = [seq[:k] for seq in dna]
    first_seq = dna[0]
    for i in range(len(first_seq) - k + 1):
        motifs = [first_seq[i:i + k]]
        for j in range(1, t):
            profile = compute_profile(motifs, k, len(motifs))
            next_motif = profile_most_probable_kmer(dna[j], k, profile)
            motifs.append(next_motif)
        if score(motifs, k) < score(best_motifs, k):
            best_motifs = motifs
    return best_motifs

# Example usage:
k, t = 3, 5
dna = ["GGCGTTCAGGCA", "AAGAATCAGTCA", "CAAGGAGTTCGC", "CACGTCAATCAC", "CAATAATATTCG"]
print(" ".join(greedy_motif_search(dna, k, t)))


CAG CAG CAA CAA CAA
